# Forest Fire Propagation

## Elements 

A forest is represented by an n*m grid, where each cell has one of the following possible types:

- WATER
- GRASS
- DENSE_TREES
- FOREST
- BURNING
- BURNED

* BURNING cells can only become BURNED
* WATER and BURNED cells remain the same
* the othe types of cells can become BURNING

In [1]:
from enum import Enum

class CellType(Enum):
    BURNED = 0
    GRASS = 1
    DENSE_TREES = 2
    FOREST = 3
    BURNING = 4
    WATER = 5

Every type of cell has a predefined color, stored in the dictionray `cell_colors`

In [2]:
cell_colors = { 
    CellType.BURNED: 'black',
    CellType.GRASS: 'springgreen',
    CellType.DENSE_TREES: 'green',
    CellType.FOREST: 'forestgreen',
    CellType.BURNING: 'crimson',
    CellType.WATER: 'royalblue'
    }

Each type of cell has a different probability of igniting based on the type of vegetation. These probabilities are stored in the dictionary `igniting_probabilities`.
Each cell also has a very small probability of self ignite, stores in constant `self_ignite`

In [3]:
igniting_probabilities = { 
    CellType.BURNED: 0,
    CellType.GRASS: 0.2,
    CellType.DENSE_TREES: 0.4,
    CellType.FOREST: 0.3,
    CellType.BURNING: 0,
    CellType.WATER: 0
    }

self_ignite = 0.0001

Based on the number of burning neighbors, the probability of a cell igniting is multiplied by a factor, stored in `burning_neighbours_factor`.
It's an array where, at position 0, there is the multiplicative factor that will be multiplied if I have 0 burning neighbours, up until a maximum of 8 neighbours.


In [4]:
burning_neighbours_factor = [1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8] 

Other constants that are required for the additional features are the maximum possible elevation of the forest, stored in `max_elevation` and the maximum speed of the wind, stored in `max_wind_speed`. Also a dictionary, mapping from String to 2-tuple to make the wind-setting more intuitive :)

In [5]:
max_elevation = 8000
max_wind_speed = 10
wind_factor = 1
wind_directions = {
    "South": (-1, 0),
    "North": (1, 0),
    "East": (0, 1),
    "West": (0, -1),
    "South-East": (-1, 1),
    "South-West": (-1, -1),
    "North-East": (1, 1),
    "North-West": (1, -1),
    "None": (0, 0)
}

## 1.a `initialize_grid` function

Outputs a `m x n` grid, which represents the forest. Each cell has a specific i,j coordinate and a type. 
In particular, the grid will be a grid of `CellTypes`, enum defined above.
At first, I store the possible types of cells in an array so i can use function random.choices.
Because we want the initial grid to be randomly generated, we have a set of probability weights, these weights determine the likelihood of a cell to be of one of the possible types. In the beginning, no cell is burned already

### `initialize_grid` pseudocode

In [6]:
# INITIALIZE_GRID(m, n)
# 1. Initialize an array of cell types
# 2. Create the array of weights
# 3. Generate a list of k elements, where each element is randomly chosen from the list of cell_types, where the probability of choosing each element is given by the list of weights
# 4. Shape the list into a matrix of size n x m
# 5. Return the grid

### `initialize_grid` code


In [7]:
import random
import numpy as np

def initialize_grid(n,m):
    # the possible types of cells in an array so i can use function random.choices
    cell_types = [CellType.BURNED, CellType.GRASS, CellType.DENSE_TREES, CellType.FOREST, CellType.BURNING, CellType.WATER]

    # because we want the initial grid to be randomly generated, we have a set of probability weights, 
    # these weights determine the likelihood of a cell to be of one of the possible types
    # in the beginning, no cell is burned already
    weights = [0, 0.3, 0.2, 0.3, 0.000001, 0.005]

    # the grid is an nxm matrix (represented as an array of arrays)
    grid_size = n * m

    grid1 = random.choices(cell_types, weights, k = grid_size)
    grid2 = np.array(grid1)
    grid = grid2.reshape((n, m))
    grid[n//2, m//2] = CellType.BURNING
    return grid

## 1.b  `plot_grid` function

Visualize the current state of the grid using different colors for each type of area using the PIL library.
The pseudocode represents is the logic i would use, the actual plot_grid function is implemented using Pillow

### `plot_grid` pseudocode

In [8]:
# PLOT_GRID(grid)
# 1. for each cell, turn it into a square with size b x h
#   i. color the square with the color of the cell
# 2. initialize a container for the rows
# 3. for each row in the grid:
#   i. rows.append(some_horizontal_combinator(row)) where some_horizontal_combinator is a function that takes an array and combines all its elements (that are graphics) horizontally and returns a single image
# 4. variable result = some_vertical_combinator(rows) where some_vertical_combinator is a function that takes an array and combines all its elements (that are graphics) vertically and returns a single image
# 5. return result, which is an image

### `plot_grid` code

In [9]:
from PIL import Image, ImageDraw

# agnese gave me this function :)

def plot_grid(grid):
    n, m = grid.shape
    # Create a blank image
    cell_size = 20
    image = Image.new("RGB", (cell_size * m, cell_size * n))

    # Create a drawing context
    draw = ImageDraw.Draw(image)
   
   # Draw the grid
    for i in range(n):
        for j in range(m):
            value = grid[i, j]
            cell_color = cell_colors[value]
            draw.rectangle(
                [j * cell_size, i * cell_size, (j + 1) * cell_size, 
                (i + 1) * cell_size],
                fill=cell_color
            )
            
    return image

## 1.c `neighbours` function

Given the coordinates of a cell and the grid, returns an array of coordinates of all its neighbours.

### `neighbours` pseudocode

In [10]:
# NEIGHBOURS(x, y, n, m)
# 1. initialize the result container
# 2. if the point is in not in the grid, return error
# 3. else:
#   i. create an array of tuples with the coordinates of the possible neighbors (8 possibilities)
#   ii. for each pair of coordinates:
#       a. compute the coordinates of the neighbor
#       b. if the neighbor is in the grid, add it to the result container
#   iii. return the result container

### `neighbours` code

In [11]:
def neighbors(grid, x, y):
    n, m = grid.shape

    # initialize result container
    result = [] 

    # check is the given point is in the size of the grid
    if (x > m or y > n) and (x < 0 or y < 0): 
        return "Error: x or y out of bounds"
    
    else:
        offsets = [(1,0), (0,1), (-1,0), (0,-1), (1,1), (-1,-1), (1,-1), (-1,1)]

        for i,j in offsets:

            # calculate new point based on given offsets and check validity
            if (x+i >= 0 and x+i < m) and (y+j >= 0 and y+j < n): 

                # if new point is valid: it's a neighbor!! append it to the result 
                result.append((x+i, y+j)) 
                
        return result

## 1.d `propagate` function

Given the grid and a cell coordinate, determines the type of the cell at the next tick of time (t+1).

### `propagate` pseudocode

In [12]:
# PROPAGATE(grid, x, y, environmental_factors)
#   1. get the type of cell at coordinates x,y in the grid
#   2. if the type of the cell is burning, return CellType.BURNED
#   3. compute the neighbors of the cell
#   4. check in the grid if the neighorsare burning are burning and count them
#   5. compute the probability of the given cell burning based on the number of burning neighbors and the type of the cell and environmental_factors
#   6. generate number with random.random() a number between 0 and 1
#   7. check if the number is smaller than the probability of the cell burning 
#   8. if yes: return CellType.BURNING, 
#   9. else: return current cell

### `propagate` code

In [13]:
def propagate(grid, x, y, wind, wind_speed, elevation):
    # get the wind vector
    wind_vector = wind_directions[wind]
    
    # get the current cell
    current = grid[x, y]
    
    # use the global variable wind_factor
    global wind_factor

    # if the cell is already burning, it will be burned
    if current == CellType.BURNING:
        return CellType.BURNED
    elif current == CellType.WATER or current == CellType.BURNED:
        # if the cell is water or burned the state will remain the same
        # (it cannot catch fire)
        return current
    
    # get the neighbors of the cell
    neighbors_list = neighbors(grid, x, y) 
    total_neighbors = len(neighbors_list) 

    # how many neighbors are burning
    burning_neighbors = 0
    # how many neighbors are water
    water_count = 0 

    for i,j in neighbors_list: 
        if grid[i, j] == CellType.BURNING: 
            burning_neighbors += 1 
            # check if the burning cell is in the direction of the wind
            if (i - x, j - y) == wind_vector:
                wind_factor = (1.2)**wind_speed
            if (x-i, y -j) == wind_vector:
                wind_factor = 1/(1.2)**wind_speed       
        # check if the neighbor is water
        if grid[i, j] == CellType.WATER:
            water_count += 1        
    # arbitrarily defined environmental factors
    water_factor = 1 - (water_count / total_neighbors)
    elevation_factor = 1- (elevation / (max_elevation + 1))
    # if there are no burning neighbors, the probability of ignition depends on the self igniting property of a cell and environmental factors
    if(burning_neighbors == 0): 
        # the probability of catching fire is self_ignite (see above) and environmental factors
        p = self_ignite * elevation_factor * wind_factor * water_factor
    # if the cell has burning neighbors
    else: 
        # probability p of burning is calculated based on arbitrary defined environmental factors (see above)
        p = igniting_probabilities[current] * burning_neighbours_factor[burning_neighbors] * wind_factor * elevation_factor * water_factor
    # if the random number is smaller than the probability, the cell will catch fire (i.e if )
    if random.random() < p:
        return CellType.BURNING 
    # if the cell is either burned or water, it will stay the same
    else: 
        return current

## 1.e `update_grid` function
Takes the grid and the environmental parameters, returns a new, updated grid using the propagate function
### `update_grid` pseudocode

In [14]:
# UPDATE_GRID(grid, environmental_factors)
# 1. get the grid's dimensions
# 2. generate an empty grid "new_grid" with the same dimensions as "grid"
# 3. fill each cell of "new_grid" with the value returned from "propagate(grid, i, j)"
# 4. return the new grid

### `update_grid` code

In [15]:
def update_grid(grid, wind, wind_speed, elevation):
    n, m = grid.shape
    new_grid = [CellType.BURNED] * (n*m)
    new_grid = np.array(new_grid).reshape((n, m))

    for i in range(n):
        for j in range(m):
            new_grid[i][j] = propagate(grid, i, j, wind, wind_speed, elevation)

    return np.array(new_grid)

In [16]:
def are_cells_burning(grid):
    # get the size of the grid
    n,m = grid.shape

     # loop through the grid
    for i in range(n):
        for j in range(m): 

            # if there is a burning cell, return True
            if grid[i,j] == CellType.BURNING: 
                return True
            
    return False

## 1.f `simulate` function
the logic i would use is in the pseudocode, the actual function is implemented in a more complex way but the core logic is the same

### `simulate` pseudocode

In [17]:
# SIMULATE(grid, envrionmental_factors)
# while cells_are_burning in the grid:
#   plot_grid(grid)
#   update_grid(grid, envrionmental_factors)

## `simulate` code

In [18]:
# agnese gave me this function :) 
import os

def simulate(n=100, m=100, wind = "None", wind_speed = 0, elevation = 0):
    grid = initialize_grid(n, m)
    frame_delay = 300
    output_dir = "animation_frames"
    os.makedirs(output_dir, exist_ok=True)

    frames = []
    frame_paths = []
    i = 0
    # generate 100 frames max
    
    while are_cells_burning(grid):
        # Draw the grid
        frames.append(grid)
        plot_image = plot_grid(grid)
        frame_path = os.path.join(output_dir, f"frame_{i:03d}.png")
        plot_image.save(frame_path)
        frame_paths.append(frame_path)
        # update the grid passing desired parameters
        grid = update_grid(grid, wind, wind_speed, elevation)
        i += 1
    # create gif
    output_gif_path = "forest_fire_animation.gif"
    frames = [Image.open(frame_path) for frame_path in frame_paths]
    frames[0].save(
    output_gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=frame_delay,
    loop=0  # 0 means an infinite loop
    )
    print("done :)")
    

## 1.g The Forest Fire Model

Run the following cell to simulate the forest fire over a 50 * 50 grid. See the resulting animation in the generated file `"forest_fire_animation.gif"`.

In [19]:
# first 2 parameters are size of the grid
# WIND is one of: "North", "South", "West", "East", "North-East", "North-West", "South-East", "South-West", "None
# WIND_SPEED is a number between 0 and 10 included (beware, 10 is VERY STRONG)
# ELEVATION is a number between 0 and 8000 included


simulate(50, 50)

done :)


## 2.a 

**QUESTION:**

Identify and explain the random variables involved in the `update_grid` function.
How are these random variables utilized in the simulation?

**ANSWER:**

The random variable involved in updating the grid is actually used in the propagate function.

Each cell has one of the following types: BURNED, GRASS, DENSE_TREES, FOREST, BURNING or WATER.

Each one of these types has an arbitrary number between 0 and 1 associated to is: this number represents the likelihood *p* of the cell catching fire.
i.e. water has 0, burned also has 0 because a burned patch of forest does not "reburn", dense_trees hase a higher number than forest etc... 

Based on the numbers of burning neighbors I have created a factor, which is then multiplied to *p*, increasing it. 
This means: if I have a lot of burning neighbors, it will be more likely that I catch fire.
Therefore, the probability that i catch fire depends on the status of my neighboring cells.
As stated in the assignment: cells also have a certain probability *self_ignition* of self igniting.
The value of this probability is arbitrarily chosen by me and it's small.

I generate a random number between 0 and 1 and there are two random variable functions that map in the following way:
if *random_numer* < *p*, then the cell will be changed to burning.
Otherwise, it will, if *random_number* < *self_ignition*, ignite by itself, or remain the same.

So the the first RV maps a randomly generated value to one of two outcomes: cell ignites because of neighbors or 
going through the second RV: cell ignites based on *self_ignition*.

## 2.b

**QUESTION:**

Describe the role of random number generators in the `update_grid` function. How are random numbers generated and utilized in the simulation?

**ANSWER:**

As decribed above, the random generator is used to determine if a cell will burn or not. 

I also randomly generate the initial grid in the following way:

We want to select n random elements from a sequence of possible cell types with replacement (because the events are independent - meaning: if the previous cell is GRASS, this has no inpact on the next one being seomething in particular - at every cell-creation we consider all possible cell types).

This is done by generating a *random_number* between 0 and 1.
This *random_number* will be used to select an element based on its weight, by accumulating the weights of all options and comparing them with the *random_number*.
i.e. with weights = [0, 0.3, 0.2, 0.3, 0.1, 0.1], if my *random_number* is 0.6, it will be mapped to the cell type that corresponds to the third type because (0 + 0.3 + 0.2) < 0.6 < (0 + 0.3 + 0.2 + 0.3).

This is done by using random.choices() with weighted sampling (sampling with specified probabilities).


## 2.c

**QUESTION:**

Explain the stochastic processes involved in the `update_grid` function. How do these processes influence the behavior of the fire propagation?

**ANSWER:**

Stochastic elements are in both update_grid and propagate functions. 
They are introduced through the use of random numbers. 
These random numbers are utilized to determine whether a cell changes state (e.g., from unburned to burning) based on probabilities associated with the cell's type, its neighbors, and other factors.
The propagate function determines whether a cell at position (x, y) in the grid should change its state (e.g., from GRASS to BURNING) based on probabilistic calculations: it checks the type of the current cell, computes the neighbors of the cell, and counts how many of those neighbors are in a burning state. It then calculates the probability of the cell catching fire, taking into account the type-specific ignition probability and the number of burning neighbors. Finally, it generates a random number to compare with the calculated probability to decide whether the cell ignites or remains the same.
The outcome of this comparison determines whether the cell changes its state, adding a stochastic element to the simulation.

Random numbers are also used during the initial grid generation, where the random.choices() function is employed to randomly assign cell types with specified probabilities.


## 2.d

**QUESTION:**

Does `update_grid` exhibit Markov Chain characteristics? How many Markov chains are involved? Explain.

**ANSWER:**

The process of updating the grid in this fire simulation does exhibit Markov chain characteristics. 
A Markov chain is a stochastic process where the future state depends only on the current state and is independent of the past states, given the current state. 

In this simulation:
Each cell's state (e.g., BURNING, GRASS, etc.) at time step t depends only on its current state at time step t-1 and the states of its neighbors at time step t-1.
The future state of each cell is determined probabilistically based on its current state and the states of its neighbors.
These transition probabilities are determined independently for each cell in the grid based on its current state and the states of its neighbors.
The simulation iteratively updates the grid over time, and the state of each cell at each time step depends only on the previous time step's state, adhering to the Markov property.
Regarding the number of Markov chains involved, there is one Markov chain for each cell in the grid. Each cell undergoes state transitions independently based on its own characteristics and the states of its neighbors. Therefore, there are a total of grid_rows x grid_columns Markov chains in this simulation.

## 3.a

**QUESTION:**

Discuss the potential influence of wind speed and direction on fire propagation. In particular, propose modifications to the transition probabilities to include this factor.

**ANSWER:**

Wind direction could impact the propagation by increasing the probability that cells located in the given direction ignite, and the speed as well.
Wind direction could be represented as a vector.
Depending on the wind direction, cells upwind would have a lower chance of catching fire from a burning cell, while those downwind would have a higher chance.
The higher the wind speed, the greater the probability that the fire will spread.


## 3.b

**QUESTION:**

Explore the effect of different vegetation types on fire spread. In particular, propose modifications to the transition probabilities to include this factor.

**ANSWER:**

I arbitrarily decided that some vegetations is at higher risk of ignition. It's explained in the questions above.
They are represented with different colors: 
```python 
CellType.GRASS: 'springgreen',
CellType.DENSE_TREES: 'green',
CellType.FOREST: 'forestgreen'
```

They also have different probabilities of igniting:
```python
igniting_probabilities = { 
    CellType.BURNED: 0,
    CellType.GRASS: 0.4,
    CellType.DENSE_TREES: 0.2,
    CellType.FOREST: 0.3,
    CellType.BURNING: 0,
    CellType.WATER: 0
    }
```

Grass has more probability of igniting due to the higher concentration of oxygen in fields and the type of vegetation being more delicate.
Dense trees have less probability of igniting because of the water content of the trees and the lower oxygen and higher humidity.
Regular forest is somwehere in between.
I, a certified botanist, am 100% sure about the above information, therefore the simulation is 100% accurate :P



## 3.c

**QUESTION:**

Discuss how terrain elevation could affect fire spread. In particular, propose modifications to the transition probabilities to include this factor.

**ANSWER:**

Greater elevation means lower oxygen, which means less probability of the fire spreading. We could apply some scalar (mutiplicative factor) that reduces che probability all the cells igniting based on the elevation of the entire forest.

## 3.d

**QUESTION:**

Identify and discuss at least one other environmental or external factor that could affect forest fire propagation. In particular propose how this factor modifies the transition probabilities.

**ANSWER:**

The presence of water-neighbors means less probability of the fire spreading. We could apply some scalar (mutiplicative factor) that reduces the probability of a cell igniting based on the number of water neighbors. Propose how this factor modifies the transition probabilities.

## 3.e

**QUESTION:**

Implement the modifications discussed in the previous section to your Forest Fire Model. Ensure your implementation is able to simulate the fire propagation with the incorporated factors and demonstrate the working model. Explain any deviations from the initial ideas and the reasons behind such changes. Your code should compile and run correctly to demonstrate the modified model.

**ANSWER:**

The modifications are already implemented in the provided code and pseudocode :)
